In [1]:
import pandas as pd

In [3]:
absenteeism_2 = 'Resources/absenteeism.csv'

absenteeism_2df = pd.read_csv(absenteeism_2, index_col=False)

absenteeism_2df.head()

,Unnamed: 0,EmployeeNumber,Gender,City,JobTitle,DepartmentName,StoreLocation,Division,Age,LengthService,AbsentHours
0,0,1,F,Burnaby,Baker,Bakery,Burnaby,Stores,32.0,6.02,36.58
1,1,2,M,Courtenay,Baker,Bakery,Nanaimo,Stores,40.0,5.53,30.17
2,2,3,M,Richmond,Baker,Bakery,Richmond,Stores,49.0,4.39,83.81
3,3,4,F,Victoria,Baker,Bakery,Victoria,Stores,45.0,3.08,70.02
4,4,5,M,New Westminster,Baker,Bakery,New Westminster,Stores,36.0,3.62,0.00
